In [ ]:
#from jax.config import config
#config.update("jax_enable_x64", True)
import numpy as np
import anndata
import pandas as pd
import scanpy as sc
import scipy 
import seaborn as sns
import matplotlib.pyplot as plt
import os
import time
#os.environ["JAX_PLATFORM_NAME"] = "cpu"
from moscot.problems.time._lineage import TemporalProblem
import warnings
import jax
import jax.numpy as jnp
from ott.geometry import pointcloud
from ott.problems.linear import linear_problem
from ott.solvers.linear import sinkhorn, sinkhorn_lr
import sys,os
sys.path.append('/home/icb/manuel.gander/mouse_atlas/notebook')
import scripts as scr
import c2
warnings.simplefilter(action='ignore', category=FutureWarning) 
sc.settings.verbosity = 0
import wandb

In [ ]:
Path="/home/icb/manuel.gander/mouse_atlas/data"

In [ ]:
ts=['E3.5', 'E4.5', 'E5.25', 'E5.5', 'E6.25', 'E6.5', 'E6.75', 'E7.0', 'E7.25', 'E7.5', 'E7.75', 'E8.0', 'E8.25', 'E8.5a', 'E8.5b', 'E9.5', 'E10.5', 'E11.5', 'E12.5', 'E13.5']
cells=[32, 80, 100, 120, 400, 660, 1720, 4500, 8200, 15000, 30000, 60000, 73000, 90000, 90000, 200000, 1100000, 2600000, 6000000, 13000000]
Cell_number_dict={ts[i]:cells[i] for i in range(20)}

In [ ]:
def solve_moscot_and_save_CT(i, epsilon, tau1, batch_size=10**5):
    tau2=0.99995
    
    ts0=ts[i]
    ts1=ts[i+1]
    print(f'{ts0}_{ts1}')

    adata=sc.read(f"{Path}/Comb_anndatas/adata_{ts0}_{ts1}.h5ad")
    del adata.raw
    adata.obs['day']=adata.obs['day'].astype('category')
    day0,day1=sorted(set(adata.obs['day']))

    tp=TemporalProblem(adata)
    if i!=13:
        tp.score_genes_for_marginals(gene_set_proliferation='mouse',  gene_set_apoptosis='mouse')
    tp = tp.prepare('day', joint_attr=f'X_pcaS')
    
    result=tp.solve(batch_size=batch_size, epsilon=epsilon, tau_a=tau1, tau_b=tau2, rank=rank)

    
    gr=tp[(day0, day1)].solution.a
    gr=gr/gr.mean()*Cell_number_dict[ts1]/Cell_number_dict[ts0]
    cell_dying=np.sum((1-gr[gr<1]))
    apoptosis_rate=float(cell_dying/len(gr))
    print(f'{ts0}: {apoptosis_rate}')
    
    cell_states0={'cell_state': list(set(adata[adata.obs['day']==day0].obs['cell_state']))}
    cell_states1={'cell_state': list(set(adata[adata.obs['day']==day1].obs['cell_state']))}
    CT=tp.cell_transition(day0, day1, cell_states0, cell_states1)
    CT.to_pickle(f'{Path}/moscot_maps/Anchors/CTs/{ts0}_{ts1}_cell_type_transitions.pkl')

    return(CT)

In [ ]:
# Save the result to Weights-and-Biases

In [ ]:
epsilon=0.005
batch_size=3000

In [ ]:
# I determined these taus such that the apoptosis rate is in the appropraite range for the different developmental stages
tau1s=[0.99,0.98,0.99,0.92,0.985,0.9,0.3,0.65,0.9,0.82,0.93,0.996,0.96,0.99,0.98,0.68,0.8,0.9,0.9]

In [ ]:
for i in range(18):
    solve_moscot_and_save_CT(i, epsilon, tau1s[i], batch_size):